In [10]:
from pydantic import BaseModel, field_serializer

In [34]:
from datetime import datetime

In [16]:
class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer("dt", when_used = "always")
    def serialize_name(self, value):
        print(f"type={type(value)}")
        return value;

In [17]:
m=Model(dt="2024-01-09")

In [18]:
m.model_dump()

type=<class 'datetime.datetime'>


{'dt': datetime.datetime(2024, 1, 9, 0, 0)}

In [20]:
m.model_dump_json()

type=<class 'datetime.datetime'>


'{"dt":"2024-01-09T00:00:00"}'

In [24]:
class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer("dt", when_used = "unless-none")
    def serialize_name(self, value):
        print(f"type={type(value)}")
        return value;

In [30]:
m=Model(dt="2024-01-09")
m.model_dump_json()

type=<class 'datetime.datetime'>


'{"dt":"2024-01-09T00:00:00"}'

In [31]:
m = Model(dt=None)

m.model_dumo(

In [33]:
m.model_dump()

{'dt': None}

In [35]:
dt = datetime(2024,9,9,0,0)

In [37]:
dt.isoformat()

'2024-09-09T00:00:00'

In [39]:
dt.strftime("%Y/%-m/%-d %I:%M %p")

'2024/9/9 12:00 AM'

In [42]:
    dt: datetime | None = None

    @field_serializer("dt", when_used = "unless-none")
    def serialize_name(self, value):
        print(f"type={type(value)}")
        return value;

m=Model(dt="2024-01-09")
m.model_dump_json()

In [43]:
m=Model(dt="2024-01-09")
m.model_dump_json()

type=<class 'datetime.datetime'>


'{"dt":"2024-01-09T00:00:00"}'

In [44]:
m.model_dump()

type=<class 'datetime.datetime'>


{'dt': datetime.datetime(2024, 1, 9, 0, 0)}

In [45]:
m=Model(dt=None)

In [46]:
m.model_dump()

{'dt': None}

In [47]:
m.model_dump_json()

'{"dt":null}'

In [48]:
from pydantic import FieldSerializationInfo

In [49]:
class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer("dt", when_used = "unless-none")
    def dt_serializer(self, value, info:FieldSerializationInfo):
        print(f"info={info}")
        return value;

In [51]:
m=Model(dt="2024-01-09")
m

Model(dt=datetime.datetime(2024, 1, 9, 0, 0))

m.model_dumo()

In [52]:
m.model_dump()

info=SerializationInfo(include=None, exclude=None, context=None, mode='python', by_alias=False, exclude_unset=False, exclude_defaults=False, exclude_none=False, round_trip=False, serialize_as_any=False)


{'dt': datetime.datetime(2024, 1, 9, 0, 0)}

In [53]:
m.model_dump_json()

info=SerializationInfo(include=None, exclude=None, context=None, mode='json', by_alias=False, exclude_unset=False, exclude_defaults=False, exclude_none=False, round_trip=False, serialize_as_any=False)


'{"dt":"2024-01-09T00:00:00"}'

In [61]:
class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer("dt", when_used = "unless-none")
    def dt_serializer(self, value, info:FieldSerializationInfo):
        print(f"info={info}")
        print(f"is_json={info.mode_is_json()}")
        return value

In [62]:
m=Model(dt="2024-01-09")

In [63]:
m.model_dump()

info=SerializationInfo(include=None, exclude=None, context=None, mode='python', by_alias=False, exclude_unset=False, exclude_defaults=False, exclude_none=False, round_trip=False, serialize_as_any=False)
is_json=False


{'dt': datetime.datetime(2024, 1, 9, 0, 0)}

In [64]:
m.model_dump_json()

info=SerializationInfo(include=None, exclude=None, context=None, mode='json', by_alias=False, exclude_unset=False, exclude_defaults=False, exclude_none=False, round_trip=False, serialize_as_any=False)
is_json=True


'{"dt":"2024-01-09T00:00:00"}'

In [65]:
import pytz

In [70]:
dt=datetime.now(pytz.utc)
dt

datetime.datetime(2024, 12, 6, 4, 27, 49, 355297, tzinfo=<UTC>)

In [71]:
dt.isoformat()

'2024-12-06T04:27:49.355297+00:00'

In [74]:
def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    
    return dt

In [78]:
dt = make_utc(datetime.now(pytz.utc))

In [79]:
dt

datetime.datetime(2024, 12, 6, 4, 32, 0, 784269, tzinfo=<UTC>)

In [80]:
dt.strftime("%Y/%-m/%-dT%H:%M:%SZ")

'2024/12/6T04:32:00Z'

In [83]:
def dt_utc_json_serializer(dt: datetime) -> str:
    dt = make_utc(dt)
    return dt.strftime("%Y/%-m/%-dT%H:%M:%SZ")

In [85]:
class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer("dt", when_used = "unless-none")
    def dt_serializer(self, value, info:FieldSerializationInfo):
        if info.mode_is_json():
            return dt_utc_json_serializer(dt)
        return make_utc(dt)

In [88]:
m=Model(dt=datetime(2020,1,1))
m

Model(dt=datetime.datetime(2020, 1, 1, 0, 0))

In [89]:
m.model_dump()

{'dt': datetime.datetime(2024, 12, 6, 4, 39, 8, 346567, tzinfo=<UTC>)}

In [90]:
m.model_dump_json()

'{"dt":"2024/12/6T04:39:08Z"}'

In [91]:
eastern = pytz.timezone("US/Eastern")
dt = eastern.localize(datetime(2024,1,1))
dt

datetime.datetime(2024, 1, 1, 0, 0, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>)

In [92]:
dt.isoformat()

'2024-01-01T00:00:00-05:00'

In [93]:
m=Model(dt=dt)
m

Model(dt=datetime.datetime(2024, 1, 1, 0, 0, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>))

In [94]:
m.model_dump()

{'dt': datetime.datetime(2024, 1, 1, 5, 0, tzinfo=<UTC>)}

In [95]:
m.model_dump_json()

'{"dt":"2024/1/1T05:00:00Z"}'